# Data Acquisition


## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
import os
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

## Loading Outcomes

In [14]:
def load_outcomes (path):
    pd.set_option('display.max_columns', None)

    path_to_file = path
    outcomes = pd.read_csv(path_to_file, encoding='ISO-8859-1', sep=';', index_col=0)

    # Replacing No and Yes by 0 and 1. 
    outcomes = outcomes.replace({'No': 0})
    outcomes = outcomes.replace({'Yes': 1})


    # Combine right and left columns
    outcomes ["CDP_25lcva_RL"] = outcomes.apply(lambda x: x.CDP_25lcva_right or x.CDP_25lcva_left, axis=1)
    outcomes ["CDP_25lcva_7L_RL"] = outcomes.apply(lambda x: x.CDP_25lcva_right_sevenletters or x.CDP_25lcva_left_sevenletters, axis=1)

    # Remove _change columns except for "EDSS_change"
    excluded_columns = ["_change", "X9HPT_change", "SDMT_change", "CDP_25lcva_right", "CDP_25lcva_left", 
                        "CDP_25lcva_right_sevenletters", "CDP_25lcva_left_sevenletters",
                       "X25LCVA_change_right", "X25LCVA_change_left", "rrms_to_spms", "Gd_lesions_at_followup"] 
    outcomes = outcomes[outcomes.columns.drop(excluded_columns)]
    outcomes = outcomes.rename_axis("Id")
    
    # Converting to binary outcomes
    outcomes['EDSS_change'] = pd.cut(outcomes['EDSS_change'], [-2.5, 0, 4.1], labels=[0, 1], include_lowest=True)
    outcomes['new_t2_lesion'] = [1 if x>0 else 0 for x in outcomes["new_t2_lesion"] ]

    
    return outcomes


## Loading Features

In [19]:
def load_features (features_path):
    
    # Getting FEATURES
    path_to_file = features_path
    data = pd.read_csv(path_to_file, encoding='ISO-8859-1', sep=';', index_col=0)

    #Dropping Columns
    data.drop('Sys4MS Identification Number', axis=1, inplace=True)
    data.drop('Norm. Factor', axis=1, inplace=True)
    #data.drop('Event Name', axis=1, inplace=True)
    data.drop('Data Access Group', axis=1, inplace=True)
    data.drop('Etnicity/Race', axis=1, inplace=True)
    data.drop('Previous treatments', axis=1, inplace=True)
    data.drop('Comorbidities', axis=1, inplace=True)
    data.drop('ONHV OD', axis=1, inplace=True)
    data.drop('ONHV OS', axis=1, inplace=True)
    data.drop('PBVC', axis=1, inplace=True)
    data.drop(list(data.filter(regex = 'Quality')), axis = 1, inplace = True)
    data.drop(list(data.filter(regex = 'Acquisition')), axis = 1, inplace = True)
    data.drop(list(data.filter(regex = 'Complete')), axis = 1, inplace = True)
    data.drop(list(data.filter(regex = 'Response to treatment')), axis = 1, inplace = True)

    #Encode "Gender"
    data['Gender'] = data['Gender'].map({'Male':0, 'Female':1})

    #Data types 
    data['MS Onset Date'] = pd.to_datetime(data['MS Onset Date'])

    # Distinguish data from baseline or 24 months.
    old_feat = data[data['Event Name'] == 'Baseline (Arm 1: Sys4MS Mandatory)']

    # Drop null columns from new_feat and old_feat
    old_feat = old_feat.dropna(how='all', axis=1)

    #Calculating 'age' column
    import datetime
    now = datetime.datetime.now()
    # Remove patients with no Birth Date
    old_feat = old_feat.dropna(subset=["Birth Date"])
    old_feat['Age'] = now - pd.to_datetime(old_feat['Birth Date'])
    old_feat['Age'] = old_feat['Age'].astype('timedelta64[Y]')
    old_feat['Age'] = old_feat['Age'].astype(int)
    old_feat.drop('Birth Date', axis=1, inplace=True)

    #Calculating 'treatment_duration' column
    old_feat['treatment_duration'] = pd.to_datetime(old_feat['Date of visit']) - pd.to_datetime(old_feat['Treatment initiation']) 
    old_feat['treatment_duration'] = old_feat['treatment_duration'].astype('timedelta64[D]')
    old_feat.loc[(old_feat.treatment_duration.isnull())&(old_feat["Current Treatment"]=="No treatment"), "treatment_duration"] = 0

    #Calculating 'time_since_last_relapse' column
    old_feat['time_since_last_relapse'] = pd.to_datetime(old_feat['Date of visit']) - pd.to_datetime(old_feat['Last relapse initiation date'])
    old_feat['time_since_last_relapse'] = old_feat['time_since_last_relapse'].astype('timedelta64[D]')
    old_feat.drop('Treatment initiation', axis=1, inplace=True)
    old_feat.drop('Date of visit', axis=1, inplace=True)
    old_feat.drop('Last relapse initiation date', axis=1, inplace=True)
    old_feat.drop('MS Onset Date', axis=1, inplace=True)
    old_feat.drop('Event Name', axis=1, inplace=True)

    #Factorize 'Current Treatment':
    Treatment_Fact = pd.factorize(old_feat['Current Treatment'])
    old_feat['Current Treatment'] = Treatment_Fact[0]
    Treatment_labels = Treatment_Fact[1]

    #Remove 'Center':
    old_feat.drop('Center', axis=1, inplace=True)

    # Delete RIGHT and LEFT
    #9HPT
    col = old_feat.loc[: , "9HPT right hand (sec)":"9HPT Left hand (sec)"]
    old_feat["9HPT"] = col.mean(axis=1)
    old_feat.drop('9HPT right hand (sec)', axis=1, inplace=True)
    old_feat.drop('9HPT Left hand (sec)', axis=1, inplace=True)
    #SL25
    col = old_feat.loc[: , "2.5LCVA, right eye":"2.5LCVA, left eye"]
    old_feat["SL25"] = col.mean(axis=1)
    old_feat.drop('2.5LCVA, right eye', axis=1, inplace=True)
    old_feat.drop('2.5LCVA, left eye', axis=1, inplace=True)
    #HCVA
    col = old_feat.loc[: , "HCVA LogMar Right eye":"HCVA LogMar Left eye"]
    old_feat["HCVA"] = col.mean(axis=1)
    old_feat.drop('HCVA LogMar Right eye', axis=1, inplace=True)
    old_feat.drop('HCVA LogMar Left eye', axis=1, inplace=True)
    #pRNFL
    col = old_feat.loc[: , "pRNFL OD":"pRNFL OS"]
    old_feat["pRNFL"] = col.mean(axis=1)
    old_feat.drop('pRNFL OD', axis=1, inplace=True)
    old_feat.drop('pRNFL OS', axis=1, inplace=True)
    #mRNFL
    col = old_feat.loc[: , "mRNFL OD":"mRNFL OS"]
    old_feat["mRNFL"] = col.mean(axis=1)
    old_feat.drop('mRNFL OD', axis=1, inplace=True)
    old_feat.drop('mRNFL OS', axis=1, inplace=True)
    #Temporal pRNFL 
    col = old_feat.loc[: , "Temporal pRNFL OD":"Temporal pRNFL OS"]
    old_feat["Temporal pRNFL"] = col.mean(axis=1)
    old_feat.drop('Temporal pRNFL OD', axis=1, inplace=True)
    old_feat.drop('Temporal pRNFL OS', axis=1, inplace=True)
    #TMV
    col = old_feat.loc[: , "TMV OD":"TMV OS"]
    old_feat["TMV"] = col.mean(axis=1)
    old_feat.drop('TMV OD', axis=1, inplace=True)
    old_feat.drop('TMV OS', axis=1, inplace=True)
    #TRT
    col = old_feat.loc[: , "TRT OD":"TRT OS"]
    old_feat["TRT"] = col.mean(axis=1)
    old_feat.drop('TRT OD', axis=1, inplace=True)
    old_feat.drop('TRT OS', axis=1, inplace=True)
    #GCIPL
    col = old_feat.loc[: , "GCIPL OD":"GCIPL OS"]
    old_feat["GCIPL"] = col.mean(axis=1)
    old_feat.drop('GCIPL OD', axis=1, inplace=True)
    old_feat.drop('GCIPL OS', axis=1, inplace=True)
    #INL
    col = old_feat.loc[: , "INL OD ":"INL OS"]
    old_feat["INL"] = col.mean(axis=1)
    old_feat.drop('INL OD ', axis=1, inplace=True)
    old_feat.drop('INL OS', axis=1, inplace=True)
    #ORL
    col = old_feat.loc[: , "ORL OD":"ORL OS"]
    old_feat["ORL"] = col.mean(axis=1)
    old_feat.drop('ORL OD', axis=1, inplace=True)
    old_feat.drop('ORL OS', axis=1, inplace=True)
    
    # Rename features
    old_feat.rename(columns={'uGMSSS':'MSSS',
                   'gARMSS':'ARMSS',
                   'CD3+ CD4+ CCR6-  CD161- CXCR3+ (Th1 classic)' : 'Th1 classic',
                   'CD3+ CD4+ CCR6- CD161+ CXCR3+ (Th1 non classic)' : 'Th1 non classic',
                   'CD3+ CD4+ CCR6+  CD161+ CXCR3- CCR4+  (Th17)' : 'Th17',
                   'CD3+ CD4+ CCR6+  CD161+ CxCR3 high CCR4 low  (Th1/Th17)' : 'Th1/Th17'
                  }, 
                 inplace=True)
    
    # We just need to know if the patient is under treatment or not, so we put '1' when '>0'
    old_feat['Current Treatment'] = [1 if x>0 else 0 for x in old_feat["Current Treatment"] ]

    df = old_feat
    # Remove HC and excluded from old_feat
    df = df[df["MS Subtype"] != "HC"]
    df = df[df["Patient Excluded from the study?"] != "Yes"]

    df.drop('Patient Excluded from the study?', axis=1, inplace=True)
    df.drop('Reason for excluding:', axis=1, inplace=True)
    df.drop('Was the EDSS confirmed?', axis=1, inplace=True)


    print("Size of data: ", df.shape)
    
    return df


## Join with the outcome

In [4]:
def join_outcome (df, outcomes, outcome):
    df = pd.get_dummies(df)
    df = df.join(outcomes[[outcome]])
    df = df.dropna(axis=0, subset=[outcome])
    print("Size of Data after joining with outcome: ", df.shape)
    return df


# Correlation

In [ ]:
def correlation_matrix(df):
  
    sns.set(style="white")

    # Compute the correlation matrix
    corr = df.corr()

    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(15, 10))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    
    # Draw the heatmap with the mask and correct aspect ratio
    g = sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
                square=False, linewidths=.5, cbar_kws={"shrink": .5, "orientation": "vertical"})
    g.set_yticklabels(g.get_yticklabels(), rotation = 0, fontsize = 12)
    g.set_yticklabels(g.get_xticklabels(), fontsize = 12)